In [1]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import os

In [2]:
links = {}

with open('../language-pairs.json', 'r') as f:
    pairs = json.loads(f.read())
    
    for pair in pairs:
        links[pair] = pairs[pair]['wiki']

links

{'Hindi-Persian': 'https://en.m.wiktionary.org/wiki/Category:Hindi_terms_borrowed_from_Persian',
 'English-French': 'https://en.m.wiktionary.org/wiki/Category:English_terms_borrowed_from_French'}

In [3]:
base_link = "https://en.m.wiktionary.org"

In [4]:
def get_words_ipas(dest_lang, src_lang, link, page_num, lst_invalid_words=['plural', 'not comparable']):
    borrowed_words = []

    try:
        # get list of words
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        res = soup.find_all("div", class_="mw-category-group")

        for i, r in enumerate(res):
            if i > 0:
                list_words = r.find_all('a')
                src_word = None

                for borrowed_word in list_words:

                    # for each loaned word get the wiki link and extract ipas for it.
                    ipa_page = requests.get(base_link + borrowed_word['href'])
                    ipa_soup = BeautifulSoup(ipa_page.content, 'html.parser')


                    # get the src word and wiki link to extract ipas for the src word
                    src_word_html_final = None
                    for header in ipa_soup.find_all("span", class_="mw-headline"):
                        if re.match("Etymology [1-9]",header.text):
                            try:
                                if dest_lang == header.parent.parent.find_previous_siblings()[0].find_all("span", class_="mw-headline")[0].text:
                                    if len(header.parent.find_next_siblings()) > 0:
                                        if len(header.parent.find_next_siblings()[0].find_all("span", class_="etyl")) > 0:
                                            etym = header.parent.find_next_siblings()[0].find_all("span", class_="etyl")[0]
                                            for tag in etym.find(lambda tag: tag.name == 'a'):
                                                if src_lang.lower() not in tag.text.lower():
                                                    src_word_html_final = etym.find_next("i")
                                                    meaning = header.parent.find_next_siblings('ol')[0].text.split('\n')[0]
                                                    meaning = re.sub(r'\(.+?\)',r'',meaning)
                                                    meaning = re.sub(r'\[.+?\]',r'',meaning)
                                                    meaning = meaning.replace('  ', ' ')
                                                    meaning = re.split('[.;:]',meaning)[0].strip()
                                                    src_word = src_word_html_final.text if src_word_html_final != None else ''
                                                    src_word = re.split('[,;:]',src_word)[0].strip()
                                                    if src_word in lst_invalid_words:
                                                        src_word_html_final = etym.find_next("strong")
                                                        src_word = src_word_html_final.text if src_word_html_final != None else ''
                                                        src_word = re.split('[,;:]',src_word)[0].strip()


                                                    #if len(borrowed_words) == 0 or \
                                                    #    borrowed_word.text != borrowed_words[-1][0]:
                                                    print(borrowed_word.text,src_word,tag.text.lower(),meaning)
                                                    borrowed_words.append([borrowed_word.text,src_word,tag.text.lower(),meaning]) 
                            except:
                                pass
                                            
        print(dest_lang, src_lang, page_num, "done", len(borrowed_words), "\n")
        # return borrowed_words      
        # find whether there is a next page containing more loan words if there is scrape that as well. 
        next_link = soup.find(
            lambda tag: tag.name == 'a' and 'next page' in tag.text.lower()
        )

        # return borrowed_words
        if next_link != None:
            more_words = get_words_ipas(dest_lang, src_lang, base_link + next_link['href'], page_num+1)
            borrowed_words = borrowed_words + more_words

    except Exception as e:
        # errors can be due to timeouts/connections refused due to rate limiting. can be fixed via proxies/vpns/timeouts
        print(pair, page_num, "got error", e, len(borrowed_words))
        return borrowed_words
        
    return borrowed_words

In [5]:
for pair in links:
    if os.path.exists("results/{}_false_friends.csv".format(pair)):
        overwrite = input("{}_false_friends.csv exists. Overwrite existing file? (y/n) ".format(pair))
    if not os.path.exists("results/{}.csv".format(pair)) or overwrite == "y":
        print()
        [dest, src] = pair.split('-')
        words = get_words_ipas(dest, src, links[pair], 1)
        df = pd.DataFrame(words, columns=['loan_word', 'original_word', 'other_etymology', 'other_meaning'])
        df.to_csv("results/{}_false_friends.csv".format(pair), index=False)
        print(pair, "done\n")


-a- Jak middle english Connective interfix used in forming compounds, often no longer carrying a distinct meaning
-a- Exmoor Scolding middle english And
abandon abandounen middle english To give up or relinquish control of, to surrender or to give oneself over, or to yield to one's emotions
Acre Acre portuguese A state of the North Region, Brazil
Acre Aakre norwegian A surname​
English French 1 done 5 

anime アニメ japanese An artistic style originating in, and associated with, Japanese animation, and that has also been adopted by a comparatively low number of animated works from other countries
Annam An Nam vietnamese A former colonial province of China, now part of Vietnam's present-day Tonkin
annex annexen middle english To add something to another thing, especially territory
arbor arbour middle english A shady sitting place or pergola usually in a park or garden, surrounded by climbing shrubs, vines or other vegetation
Aries aries latin A constellation of the zodiac supposedly shape

KeyboardInterrupt: 